In [9]:
from copy import deepcopy

fullList = {'constants': [], 'variables': [], 'functions': [], 'predicates': [], 'clauses': []}

class Constant:
    registry = fullList['constants']
    
    def __init__(self, s):
        self.sym = self.name(s)
        self.arity = 0
        self.length = 1
        Constant.registry.append(self)
        
    def __str__(self):
        return self.sym
        
    def __len__(self):
        return self.length
    
    def __eq__(self, other):
        return isinstance(other, Constant) and self.sym == other.sym

    def __hash__(self):
        return id(self)
    
    def name(self,s):
        if sum([1 if i.sym==s else 0 for i in Constant.registry])>0:
            return self.name(s + ".")
        else: return s
    
    def reduceSize(self):
        pass
    
    def isGround(self):
        return True
    
    def findVariables(self):
        return []
    
    def findRepeats(self):
        return [self]

    def replace(self, var, rpt):
        pass
    
class Repeat:
    def __init__(self, original):
        self.sym = '*'
        self.origin = original
        self.length = 0
        
    def __str__(self):
        #return self.sym + str(self.origin) + self.sym
        return self.sym
        
    def __len__(self):
        return self.length
    
    def __eq__(self, other):
        return isinstance(other, type(self.origin)) and \
        self.origin.sym == other.sym and self.origin.params == other.params
        
    def __hash__(self):
        return id(self)
    
    def reduceSize(self):
        pass
    
    def isGround(self):
        return self.origin.isGround()
    
    def findVariables(self):
        return self.origin.findVariables()
    
    def findRepeats(self):
        return []
    
    def replace(self, var, rpt):
        pass

    
class Variable:
    registry = fullList['variables']
    
    def __init__(self, s):
        self.sym = self.name(s)
        #self.length = 1 Use minimal possible size.
        self.length = 0
        Variable.registry.append(self)
        
    def __str__(self):
        return self.sym
    
    def __len__(self):
        return self.length
    
    def __eq__(self, other):
        return isinstance(other, Variable) and self.sym == other.sym
    
    def __hash__(self):
        return id(self)
    
    def name(self, s):
        if sum([1 if i.sym==s else 0 for i in Variable.registry])>0:
            return self.name(s + ".")
        else: return s
        
    def reduceSize(self):
        pass
    
    def isGround(self):
        return False
    
    def findVariables(self):
        return [self]
    
    def findRepeats(self):
        return [self]
    
    def replace(self, var, rpt):
        variable = deepcopy(self)
        if self==var:
                return rpt
        else:
                return

    
class Function:
    registry = fullList['functions']
    
    def __init__(self, s, *args):
        self.sym = self.name(s)
        self.params = list(args)
        self.arity = len(args)
        self.length = 1+sum([len(i) for i in self.params])
        Function.registry.append(self)

    def __str__(self):
        toReturn = self.sym + "("
        for i in self.params:
            toReturn += str(i) + ","
        toReturn = toReturn[:-1] + ")"
        return toReturn
    
    def __len__(self):
        return 1+sum([len(i) for i in self.params])
                     
    def __eq__(self, other):
        return isinstance(other, Function) and self.sym == other.sym and self.params == other.params
    
    def __hash__(self):
        return id(self)
    
    def name(self,s):
        if sum([1 if i.sym==s else 0 for i in Function.registry])>0:
            return self.name(s + ".")
        else: return s
    
    def reduceSize(self):
        for i in xrange(len(self.params)):
            for j in xrange(i+1, len(self.params)):
                if not isinstance(self.params[j], Repeat) and self.params[i] == self.params[j]:
                    self.params[j] = Repeat(self.params[j])
        for i in self.params:
            i.reduceSize()
            
    def isGround(self):
        for i in self.params:
            if not i.isGround():
                return False
        return True
    
    def findVariables(self):
        variables = []
        for i in self.params:
            variables = variables + i.findVariables()
        return variables
            
    def findRepeats(self):
        repeats = []
        for i in self.params:
            repeats = repeats + i.findRepeats()
        repeats.append(self)
        return repeats
    
    def replace(self, var, rpt):
        function = deepcopy(self)
        for i in function.params:
            if i==var:
                function.params[function.params.index(var)] = i
            else:
                i.replace(var, rpt)
        return function

class Predicate:
    registry = fullList['predicates']
    
    def __init__(self, s, *args):
        self.sym = self.name(s)
        self.params = list(args)
        self.length = 1+sum([len(i) for i in self.params])
        Predicate.registry.append(self)
        
    def __str__(self):
        toReturn = self.sym + "("
        for i in self.params:
            toReturn += str(i) + ","
        toReturn = toReturn[:-1] + ")"
        return toReturn
    
    def __len__(self):
        return 1+sum([len(i) for i in self.params])
    
    def __eq__(self, other):
        return isinstance(other, Predicate) and self.params == other.params
 
    def __hash__(self):
        return id(self)

    def name(self,s):
        if sum([1 if i.sym==s else 0 for i in Function.registry])>0:
            return self.name(s + ".")
        else: return s
        
    def reduceSize(self):
        for i in xrange(len(self.params)):
            for j in xrange(i+1, len(self.params)):
                if not isinstance(self.params[j], Repeat) and self.params[i] == self.params[j]:
                    self.params[j] = Repeat(self.params[j])
        for i in self.params:
            i.reduceSize()
            
    def findVariables(self):
        variables = []
        for i in self.params:
            variables = variables + i.findVariables()
        return variables
            
    def findRepeats(self):
        repeats = []
        for i in self.params:
            repeats = repeats + i.findRepeats()
        repeats.append(self)
        return repeats

    def replace(self, var, rpt):
        predicate = deepcopy(self)
        for i in predicate.params:
            if i==var:
                predicate.params[predicate.params.index(var)] = i
            else:
                i.replace(var, rpt)
        return predicate

class Clause:
    registry = fullList['clauses']
    
    def __init__(self, *args):
        self.params = list(args)
        self.length = sum([len(i) for i in self.params])
        Clause.registry.append(self)
        
    def __str__(self):
        toReturn = "{"
        for i in self.params:
            toReturn += str(i) + ","
        toReturn = toReturn[:-1] + "}"
        return toReturn
    
    def __len__(self):
        return sum([len(i) for i in self.params])
    
    def __eq__(self, other):
        return isinstance(other, Function) and self.sym == other.sym and self.params == other.params

    def __hash__(self):
        return id(self)
    
    def reduceSize(self):
        for i in xrange(len(self.params)):
            for j in xrange(i+1, len(self.params)):
                if not isinstance(self.params[j], Repeat) and self.params[i] == self.params[j]:
                    self.params[j] = Repeat(self.params[j])
        for i in self.params:
            i.reduceSize()    
            
    def isGround(self):
        for i in self.params:
            if not i.isGround():
                return False
        return True

    def findVariables(self):
        variables = []
        for i in self.params:
            variables = variables + i.findVariables()
        return variables
    
    def findRepeats(self):
        repeats = []
        for i in self.params:
            repeats = repeats + i.findRepeats()
        repeats.append(self)
        return repeats
    
    def replace(self, var, rpt):
        clause = deepcopy(self)
        for i in clause.params:
            if i==var:
                clause.params[clause.params.index(var)] = i
            else:
                i.replace(var, rpt)
        return clause

    
#reduceSize() could potentially be done in linear time by implementing a trie datastructure for searching. 
#Since clauses are generally relatively small, this n^2 methods should be efficient enough.
#Potentially come back to make more efficient.
#TODO: Look at objects in different higher-level objects
            
a = Constant('a')
b = Constant('a')
print str(a), str(b)
print fullList['constants']
x = Variable('x')
y = Variable('y')
a1 = Repeat(a)
f1 = Function('f', a, x)
f2 = Function('f', a, a)
g1 = Function('g', b, x)
g2 = Function('g', b, y)
g3 = Function('g', b, y)

p1 = Predicate('P',f1, f2, g2, g3)
p2 = Predicate('Q', f1)
p3 = Predicate('R', f2, g3)

c1 = Clause(p1, p2, f1)
c2 = Clause(f1)


print str(f1) + " is ground: " + str(f1.isGround())
print str(f2) + " is ground: " + str(f2.isGround())
print
print "Original Clause:" + str(c1)
print "Original Length: " + str(len(c1))
print
c1.reduceSize()
print "Modified Clause:" + str(c1)
print "Modified Length: " + str(len(c1))


for i in set(c1.findRepeats()):
    print i
    
clause = c1

a a.
[<__main__.Constant instance at 0x104250f80>, <__main__.Constant instance at 0x104250d40>]
f(a,x) is ground: False
f.(a,a) is ground: True

Original Clause:{P(f(a,x),f.(a,a),g.(a.,y),g..(a.,y)),Q(f(a,x)),f(a,x)}
Original Length: 15

Modified Clause:{P(f(a,x),f.(a,*),g.(a.,y),g..(a.,y)),Q(f(a,x)),f(a,x)}
Modified Length: 14
a
a.
g.(a.,y)
x
Q(f(a,x))
y
f.(a,*)
f(a,x)
g..(a.,y)
{P(f(a,x),f.(a,*),g.(a.,y),g..(a.,y)),Q(f(a,x)),f(a,x)}
P(f(a,x),f.(a,*),g.(a.,y),g..(a.,y))


In [25]:
constants = Constant.registry
functions = Function.registry
globalRepeats = set()
clauseSet = set()

SIZE_BOUND = 3

#repeats
def enumerateClauses(clause, size, rpts):
    global constants, functions
    repeats = rpts.union(set(clause.findRepeats()))
    if size<0:
        return
    elif size==0:
        x = None
        for x in clause.findVariables():
            for i in repeats:
                clauseSet.add(clause.replace(x,i))
        if x==None:
            clauseSet.add(clause)
    else:
        for x in clause.findVariables():
            for i in set(constants).union(set(functions)).union(set(repeats)):
                if size-len(i)>=0:
                    enumerateClauses(clause.replace(x,i), size-len(i), repeats)
        sizeCheck(SIZE_BOUND)
        
def sizeCheck(size, repeats):
    for clause in clauseSet:
        if len(clause.reduceSize()) > size:
            clauseSet.remove(clause)
        else:
            for i in clause:
                repeats.add(i)

enumerateClauses(c2, SIZE_BOUND, globalRepeats)

KeyboardInterrupt: 

In [27]:
a = [1,2,3]
b = a
a = [1,2,4]
print b

a = a + []
print a

a = Constant('a')
b = Constant('b')
x = Variable('x')
y = Variable('y')
a1 = Repeat(a)
f1 = Function('f', a, x)

print f1.params
f1.params[f1.params.index(a)] = b
print f1.params

print a
print b
b = deepcopy(a)
a.sym = 'v'
print a
print b

print set(constants).union(set(functions))

[1, 2, 3]
[1, 2, 4]
[<__main__.Constant instance at 0x160dda998>, <__main__.Variable instance at 0x160ddaa28>]
[<__main__.Constant instance at 0x10ea349e0>, <__main__.Variable instance at 0x160ddaa28>]
a
b
v
a
set([<__main__.Constant instance at 0x104692200>, <__main__.Constant instance at 0x10ea349e0>, <__main__.Function instance at 0x160ddaf80>, <__main__.Function instance at 0x1046921b8>, <__main__.Constant instance at 0x160ddae18>, <__main__.Function instance at 0x104692a28>, <__main__.Function instance at 0x104692440>, <__main__.Function instance at 0x1046924d0>, <__main__.Constant instance at 0x1046925f0>, <__main__.Constant instance at 0x160ddabd8>, <__main__.Function instance at 0x104692998>, <__main__.Constant instance at 0x160dda998>, <__main__.Function instance at 0x103f2d3f8>])
